In [5]:
!pip install tensorflow_datasets

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [7]:
import tensorflow as tf

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)


#used to fix bug in keras preprocessing scope
temp = tf.zeros([4, 32, 32, 3])  # Or tf.zeros
preprocess_input(temp)
print("processed")

processed


In [8]:
IMAGE_SIZE = (224, 224)
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000
BATCH_SIZE_PER_GPU = 96
global_batch_size = (BATCH_SIZE_PER_GPU * 1)
NUM_CLASSES = 10

Dataset code

In [9]:
def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_indices=np.zeros(len(scales)), crop_size=IMAGE_SIZE)
        # Return a random crop
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random.uniform(())

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

def normalize(input_image):
  return preprocess_input(input_image)

@tf.function
def load_image_train(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  # if tf.random.uniform(()) > 0.5:
  #   input_image = tf.image.flip_left_right(input_image)
  augmentations = [flip, color, zoom, rotate]
  for f in augmentations:
    input_image = tf.cond(tf.random.uniform(()) > 0.75, lambda: f(input_image), lambda: input_image)

  #input_image = preprocess_input(input_image)
  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

@tf.function
def load_image_test(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  #input_image = preprocess_input(input_image)

  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

In [10]:
dataset, info = tfds.load('cifar10', with_info=True)

In [11]:
for example in dataset['train'].take(1):
  print(example["label"])

tf.Tensor(3, shape=(), dtype=int64)


In [12]:
info

tfds.core.DatasetInfo(
    name='cifar10',
    version=1.0.2,
    description='The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.',
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    features=FeaturesDict({
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple layers of features from tiny images},
        institution = {},
        year = {2009}
    }""",
    redistribution_info=,
)

make the upscaled cifar dataset

In [13]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train.shuffle(buffer_size=1000).batch(global_batch_size).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


test_dataset = dataset['test'].map(load_image_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(global_batch_size).repeat()
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [14]:
model = tf.keras.models.load_model('./base_model_cifar10_vgg16.h5')

In [15]:
model.compile(optimizer=tf.optimizers.SGD(learning_rate=.01, momentum=.9, nesterov=True), loss='mse', metrics=['acc'])

In [16]:
model.evaluate(test_dataset, steps=VALIDATION_SIZE//global_batch_size)

  1/104 [..............................] - ETA: 3:18

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/block1_conv1/Conv2D (defined at <ipython-input-16-57d188ed4541>:1) ]] [Op:__inference_distributed_function_3231]

Function call stack:
distributed_function


In [ ]:
for i, layer in enumerate(model.layers):
  print(f'{i} layer {layer.name}')

build training output

In [ ]:
get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[16].output, model.layers[17].output])

generator for training replacement blocks

In [15]:
import math
class LayerBatch(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(TRAIN_SIZE // global_batch_size )
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y
    
import math
class LayerTest(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(VALIDATION_SIZE // global_batch_size )
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y

In [16]:
def build_replacement(get_output):
    inputs = tf.keras.Input(shape=get_output.output[0].shape[1::])
    X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=get_output.output[1].shape[-1], 
                                        kernel_size= (3,3),
                                        padding='Same')(inputs)
    #X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
    X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
    
    build_replacement.counter += 1
    
    X = tf.keras.layers.concatenate([inputs, X])

    X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=get_output.output[1].shape[-1],
                                        kernel_size=(3,3), 
                                        padding='Same')(X)
    #X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
    X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
    replacement_layers = tf.keras.Model(inputs=inputs, outputs=X)
    
    build_replacement.counter += 1
    
    return replacement_layers

build_replacement.counter = 0

In [17]:
replacement_layers = build_replacement(get_output)
replacement_len = len(replacement_layers.layers)
layer_train_gen = LayerBatch(get_output, train_dataset)
layer_test_gen = LayerTest(get_output, test_dataset)



In [18]:
replacement_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
sep_conv_0 (SeparableConv2D)    (None, None, None, 5 267264      input_1[0][0]                    
__________________________________________________________________________________________________
relu_0 (ReLU)                   (None, None, None, 5 0           sep_conv_0[0][0]                 
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, None, None, 1 0           input_1[0][0]                    
                                                                 relu_0[0][0]               

In [19]:

MSE = tf.losses.MeanSquaredError()

optimizer=tf.keras.optimizers.SGD(.1, momentum=.9, nesterov=True)
replacement_layers.compile(loss=MSE, optimizer=optimizer)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=5, min_lr=.0001, factor=.3, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(patience=15, min_delta=.001, restore_best_weights=True, verbose=1)
history = replacement_layers.fit(x=layer_train_gen,
                               epochs=100,
                               steps_per_epoch=TRAIN_SIZE // global_batch_size,
                               validation_data=layer_test_gen,
                               shuffle=False,
                               callbacks=[reduce_lr, early_stop],
                               validation_steps=VALIDATION_SIZE // global_batch_size,
                               verbose=1)

  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 520 steps, validate for 104 steps
Epoch 1/100
520/520 [==============================] - 104s 201ms/step - loss: 3.5055 - val_loss: 1.9272
Epoch 2/100
520/520 [==============================] - 103s 198ms/step - loss: 1.6157 - val_loss: 1.2594
Epoch 3/100
520/520 [==============================] - 103s 199ms/step - loss: 1.0008 - val_loss: 0.7671
Epoch 4/100
520/520 [==============================] - 103s 198ms/step - loss: 0.6909 - val_loss: 0.6082
Epoch 5/100
520/520 [==============================] - 102s 196ms/step - loss: 0.5452 - val_loss: 0.4998
Epoch 6/100
520/520 [==============================] - 101s 194ms/step - loss: 0.4619 - val_loss: 0.4321
Epoch 7/100
202/520 [==========>...................] - ETA: 51s - loss: 0.4286

I found this code on stackoverflow but It has some problems even though I think it is mostly on the right track

In [13]:

get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[16].output])
# add in replacement layers
print('building middle of model with replacement layers')
new_joint = tf.keras.Model(inputs=get_output.input, outputs=replacement_layers(get_output.output))

#new_joint.summary()

# build bottom of model
bottom_half = tf.keras.Sequential()
for layer in model.layers[17 + 1::]:
    bottom_half.add(layer)



print('building bottom of model')
bottom_half.build(input_shape=new_joint.output.shape)
bottom_half.summary()
print('combining model')
combined = tf.keras.Model(inputs=new_joint.input, outputs=bottom_half(new_joint.output))

combined.layers[-1].trainable=False
opt = keras.optimizers.RMSprop(lr=0.00005, decay=1e-6)
combined.compile(loss='categorical_crossentropy',
          optimizer=opt,
          metrics=['accuracy'])

NameError: name 'model' is not defined

In [ ]:
new_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
new_model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size)